### DeepLoc EDA
In this file we explore the different datasets published by DeepLoc.

In [8]:
# Library imports
from collections import defaultdict
import pandas as pd
import blosum as bl
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

#### DeepLoc 1.0 Dataset
We first explore the dataset published under the [HTU website](https://services.healthtech.dtu.dk/services/DeepLoc-1.0/).

#### DeepLoc 1.0 EDA

In [9]:
# Data file for the original DeepLoc 1.0 dataset
fasta_file_path = './data/deeploc_data_original.fasta'

In [10]:

# Define locations and their desired mapping for printing.
location_categories = {
    "Nucleus": ["Nucleus"],
    "Cytoplasm": ["Cytoplasm"],
    "Extracellular": ["Extracellular"],
    "Mitochondrion": ["Mitochondrion"],
    "Cell membrane": ["Cell.membrane"],
    "Endoplasmic reticulum (ER)": ["Endoplasmic.reticulum"],
    "Plastid": ["Plastid"],
    "Golgi apparatus": ["Golgi.apparatus"],
    "Lysosome/Vacuole": ["Lysosome/Vacuole"],
    "Peroxisome": ["Peroxisome"]
}

# Find the locations for the sequence
def normalize_location(header_line):
    header_line = header_line.lower()
    matches = set()
    for category, keywords in location_categories.items():
        for keyword in keywords:
            if keyword.lower() in header_line: # Normalize, just in case.
                matches.add(category)
    return matches

# Count the number of sequences for each location
def count_locations(fasta_file_path):
    location_counts = defaultdict(int)
    with open(fasta_file_path, 'r') as fasta_file:
        for line in fasta_file:
            if line.startswith('>'):
                matched_categories = normalize_location(line)
                for category in matched_categories:
                    location_counts[category] += 1
    return location_counts

Note: This still allows the one sequence to have multiple locations.

In [83]:
location_counts = count_locations(fasta_file_path)

# Print the counts for each location
print("Location counts:")
for location in location_categories:
    print(f"{location}: {location_counts[location]}")

Location counts:
Nucleus: 4189
Cytoplasm: 2688
Extracellular: 1973
Mitochondrion: 1510
Cell membrane: 1340
Endoplasmic reticulum (ER): 862
Plastid: 757
Golgi apparatus: 356
Lysosome/Vacuole: 321
Peroxisome: 154


In [11]:
# Count the number of sequences that only have one location
def count_single_location_only(fasta_file_path):
    location_counts = defaultdict(int)
    with open(fasta_file_path, 'r') as fasta_file:
        for line in fasta_file:
            if line.startswith('>'):
                matched_categories = normalize_location(line)
                if len(matched_categories) == 1:
                    category = next(iter(matched_categories))
                    location_counts[category] += 1
    return location_counts

Note: Only Nucleus and Cytoplasm occurred together as locations for sequences 

In [12]:
location_counts = count_single_location_only(fasta_file_path)

# Print the counts for each location
print("Location counts:")
for location in location_categories:
    print(f"{location}: {location_counts[location]}")

Location counts:
Nucleus: 4043
Cytoplasm: 2542
Extracellular: 1973
Mitochondrion: 1510
Cell membrane: 1340
Endoplasmic reticulum (ER): 862
Plastid: 757
Golgi apparatus: 356
Lysosome/Vacuole: 321
Peroxisome: 154


We decided against using this dataset as the DeepLoc 2.0 offers a dataset with partitions where sequences are correctly homology reduced to 30% identity.

#### DeepLoc 2.0/2.1 EDA

In [13]:
csv_path = './data/deeploc_data_2.1.csv'
train_path = './data/deeploc2.1_training_processed.csv'
test_path = './data/deeploc2.1_test_processed.csv'

In [14]:
# Count the number of sequences for each location in the CSV file
df = pd.read_csv(csv_path, delimiter=',')

# Columns corresponding to subcellular locations/columns in the CSV file
location_columns = [
        "Cytoplasm",
        "Nucleus",
        "Extracellular",
        "Cell membrane",
        "Mitochondrion",
        "Plastid",
        "Endoplasmic reticulum",
        "Lysosome/Vacuole",
        "Golgi apparatus",
        "Peroxisome"
        ]

def count_locations(df):

    # Count occurrences
    location_counts = df[location_columns].sum().astype(int)

    print("Location counts:")
    for location, count in location_counts.items():
        print(f"{location}: {count}")
    total = location_counts.sum()
    print(f"Total Number of sequences in dataset: {total}")


count_locations(df)


Location counts:
Cytoplasm: 9870
Nucleus: 9720
Extracellular: 3301
Cell membrane: 4187
Mitochondrion: 2590
Plastid: 1047
Endoplasmic reticulum: 2180
Lysosome/Vacuole: 1496
Golgi apparatus: 1279
Peroxisome: 304
Total Number of sequences in dataset: 35974


In [15]:
# Filter out sequences with multiple locations
def count_locations_single_label(df):

    # Filter: keep only rows with exactly 1 location
    df_single_label = df[df[location_columns].sum(axis=1) == 1]

    # Count occurrences in single-label data
    location_counts = df_single_label[location_columns].sum().astype(int)

    print("Location counts (single-label only):")
    for location, count in location_counts.items():
        print(f"{location}: {count}")
    print(f"Total: {location_counts.sum()}")

    return df_single_label

# Returns a filtered DataFrame with sequences that have only one location
df_filtered = count_locations_single_label(df)

Location counts (single-label only):
Cytoplasm: 4582
Nucleus: 5462
Extracellular: 2944
Cell membrane: 2770
Mitochondrion: 2062
Plastid: 932
Endoplasmic reticulum: 1331
Lysosome/Vacuole: 761
Golgi apparatus: 556
Peroxisome: 219
Total: 21619


In [16]:
counts = df_filtered['Membrane'].value_counts()
# Print the counts for each Membrane type
print("Membrane type counts:")
for location, count in counts.items():
    print(f"{location}: {count}")

Membrane type counts:
0.0: 15261
1.0: 6358


Note: This dataset is used for k-fold testing. Meaning that over K iterations, K-1 are used for training and 1 is used for validation. Homology across folds was >30%

Mentioned in $2.3.1

In [89]:
# In DeepLoc 1.0, they also filtered out sequences with length < 40
# Count the number of sequences with length < 40
df_filtered[df_filtered['Sequence'].str.len() < 40].shape

# No sequences with length < 40

(0, 16)

In [90]:
# Additionally they mention in $2.3.5: To decrease the training time, the maximum protein length was 1000
df[df['Sequence'].str.len() > 1000].shape


(3186, 16)

We see that 3186 samples have to be truncated. We do this as they suggest, by taking out amino-acids in the center of the sequence.

Finally, amino acids shorter than length 1000 must be padded to length 1000. Additionally, a mask must be generated to ensure the attention mechanism ignores padding.

Note: Only in the description of Figure 3 is it mentioned that for *visualization* the amino acids are padded from the middle. We assume this to be the case all the time.

In [91]:
MAX_LEN = 1000
PAD_CHAR = '-'

# Remove the center of the sequence, keeping the first and last 500.
def truncate_sequence(seq, max_len=1000):
    if len(seq) <= max_len:
        return seq
    half = max_len // 2
    return seq[:half] + seq[-half:]

# Pad the sequence in the center with a specified character
def pad_middle(seq, max_len=MAX_LEN, pad_char=PAD_CHAR):
    pad_total = max_len - len(seq)
    half_idx = len(seq) // 2
    pad_left = pad_total // 2
    pad_right = pad_total - pad_left
    return seq[:half_idx] + (pad_char * pad_left) + seq[half_idx:] + (pad_char * pad_right)

# Prepare the sequence for the the processed training dataset
def prepare_sequence(seq, max_len=MAX_LEN, pad_char=PAD_CHAR):
    if len(seq) > max_len:
        return truncate_sequence(seq, max_len)
    elif len(seq) < max_len:
        return pad_middle(seq, max_len, pad_char)
    else:
        return seq  # already length 1000

# Generate a mask for the sequence
def generate_mask(seq, pad_char='X'):
    """ The mask is a string of 0s and 1s, where 1 indicates the presence of an amino acid and 0 indicates a padding character """
    return ''.join(['0' if aa == pad_char else '1' for aa in seq])

In [92]:
# Create a new column with truncated sequences
df_filtered['PaddedSequence'] = df_filtered['Sequence'].apply(prepare_sequence)
df_filtered['Mask'] = df_filtered['PaddedSequence'].apply(generate_mask)

C:\Users\neere\AppData\Local\Temp\ipykernel_5396\1970824193.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['PaddedSequence'] = df_filtered['Sequence'].apply(prepare_sequence)
C:\Users\neere\AppData\Local\Temp\ipykernel_5396\1970824193.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Mask'] = df_filtered['PaddedSequence'].apply(generate_mask)


In [93]:
# Save to a new CSV
df_filtered.to_csv(train_path, index=False)
# Filter by partition
df_train = df_filtered[df_filtered['Partition'] != 4]  # Partitions 0–3 for training/validation
df_test = df_filtered[df_filtered['Partition'] == 4]       # Partition 4 for testing

# Save to separate CSV files
df_train.to_csv(train_path, index=False)
df_test.to_csv(test_path, index=False)

In [94]:
df_processed = pd.read_csv(train_path, delimiter=',')
print(df_processed[df_processed['Sequence'].str.len() == 1000].shape)
print(df_processed[df_processed['PaddedSequence'].str.len() == 1000].shape)
print(df_processed[df_processed['Mask'].str.len() == 1000].shape)
print(df_processed.shape)

(4, 18)
(17351, 18)
(17351, 18)
(17351, 18)


##### Converting the sequence to a feature vector using BLOSUM-62

In [95]:
matrix = bl.BLOSUM(62, default=0)
print(matrix)

# Get all the keys in the dictionary and convert them to a list
matrix_keys = list(matrix.keys())

print(matrix_keys)
print(len(matrix_keys))

BLOSUM(62, default=0, {'A': defaultdict(<function BLOSUM.__init__.<locals>.<lambda> at 0x0000021214ACBB50>, {'A': 4.0, 'R': -1.0, 'N': -2.0, 'D': -2.0, 'C': 0.0, 'Q': -1.0, 'E': -1.0, 'G': 0.0, 'H': -2.0, 'I': -1.0, 'L': -1.0, 'K': -1.0, 'M': -1.0, 'F': -2.0, 'P': -1.0, 'S': 1.0, 'T': 0.0, 'W': -3.0, 'Y': -2.0, 'V': 0.0, 'B': -2.0, 'J': -1.0, 'Z': -1.0, 'X': -1.0, '*': -4.0}), 'R': defaultdict(<function BLOSUM.__init__.<locals>.<lambda> at 0x0000021214ACB880>, {'A': -1.0, 'R': 5.0, 'N': 0.0, 'D': -2.0, 'C': -3.0, 'Q': 1.0, 'E': 0.0, 'G': -2.0, 'H': 0.0, 'I': -3.0, 'L': -2.0, 'K': 2.0, 'M': -1.0, 'F': -3.0, 'P': -2.0, 'S': -1.0, 'T': -1.0, 'W': -3.0, 'Y': -2.0, 'V': -3.0, 'B': -1.0, 'J': -2.0, 'Z': 0.0, 'X': -1.0, '*': -4.0}), 'N': defaultdict(<function BLOSUM.__init__.<locals>.<lambda> at 0x0000021218746CB0>, {'A': -2.0, 'R': 0.0, 'N': 6.0, 'D': 1.0, 'C': -3.0, 'Q': 0.0, 'E': 0.0, 'G': 0.0, 'H': 1.0, 'I': -3.0, 'L': -3.0, 'K': 0.0, 'M': -2.0, 'F': -3.0, 'P': -2.0, 'S': 1.0, 'T': 0.0, '

In [96]:
# Remove B, Z, X, J, * from the keys list as they are not standard amino acids and DeepLoc 1.0 only has 20 features.
matrix_keys = [key for key in matrix_keys if key not in ['B', 'Z', 'X', 'J', '*']]
print(len(matrix_keys))
print(matrix_keys)

20
['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']


In [97]:
def encode_sequence_with_blosum(seq):
    """
    Encode a sequence using BLOSUM62, applying the mask.
    Returns: (max_len x 20) numpy array
    """
    encoded = np.zeros((MAX_LEN, len(matrix_keys)), dtype=np.float32)
    for i, aa in enumerate(seq):
        encoded[i] = [matrix[aa][other_aa] for other_aa in matrix_keys]
        # We ensured the default was 0, so no need to use the mask.
    return encoded

In [98]:
# Test the encoding function
features = encode_sequence_with_blosum("ACDXYZ--")
print(features)

[[ 4. -1. -2. ... -3. -2.  0.]
 [ 0. -3. -3. ... -2. -2. -1.]
 [-2. -2.  1. ... -4. -3. -3.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


##### Creating a dataset which encodes sequences & returns batches for training

In [99]:
# Custom dataset class for DeepLoc
class DeepLocDataset(Dataset):
    def __init__(self, df, label_columns, matrix):
        self.sequences = df['PaddedSequence'].values # Fetch the processed sequences
        self.masks = df['Mask'].values # Fetch the masks
        self.labels = df[label_columns].values.astype(np.float32) # Convert labels to float32
        self.matrix = matrix # BLOSUM matrix
        self.max_len = MAX_LEN
        self.matrix_keys = list(matrix.keys())
        self.matrix_keys = [key for key in self.matrix_keys if key not in ['B', 'Z', 'X', 'J', '*']]
        self.num_features = len(self.matrix_keys)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        mask = self.masks[idx]
        label = self.labels[idx]
        encoded_seq = encode_sequence_with_blosum(seq)
        mask_tensor = torch.tensor([int(m) for m in mask], dtype=torch.float32)
        return torch.tensor(encoded_seq), torch.tensor(label), mask_tensor # Return the encoded sequence, label, and mask tensor

In [100]:
# Create Dataset and DataLoader
dataset = DeepLocDataset(df_filtered, location_columns, matrix)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [101]:
# Test the DataLoader
for x_batch, y_batch, mask_batch in loader:
    print("x_batch shape:", x_batch.shape) # x_batch: (B, 1000, 20)
    print("y_batch shape:", y_batch.shape) # y_batch: (B, 10)
    print("mask_batch shape:", mask_batch.shape) # mask_batch: (B, 1000)
    break


x_batch shape: torch.Size([32, 1000, 20])
y_batch shape: torch.Size([32, 10])
mask_batch shape: torch.Size([32, 1000])
